In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import subprocess


# ========================
# CONFIG CHUNG
# ========================
EDGE_DRIVER_PATH = r"C:\Users\Admin\Downloads\edgedriver_win64_\msedgedriver.exe"

DESKTOP_UA = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
              "AppleWebKit/537.36 (KHTML, like Gecko) "
              "Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0")

MOBILE_UA = ("Mozilla/5.0 (Linux; Android 11; Pixel 5) "
             "AppleWebKit/537.36 (KHTML, like Gecko) "
             "Chrome/120.0.0.0 Mobile Safari/537.36")

# ========================
# HÀM TIỆN ÍCH
# ========================
def kill_msedge_processes():
    """Tắt toàn bộ tiến trình Edge & driver."""
    os.system("taskkill /F /IM msedge.exe /T >nul 2>&1")
    os.system("taskkill /F /IM msedgedriver.exe /T >nul 2>&1")

def sync_edge_account(driver):                      
    """Tùy chọn: click Sync nếu thấy."""
    try:
        sync_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Sync')]"))
        )
        sync_btn.click()
        print("🔄 Đã click sync.")
    except:
        pass


# ========================
# CORE FUNCTION
# ========================
def auto_bing_search(keywords, profile_path, profile_name="Default", 
                     mode="desktop", use_sync=False):
    """
    Tự động tìm kiếm Bing và click kết quả đầu tiên
    mode: "desktop" hoặc "mobile"
    """
    kill_msedge_processes()

    for i, keyword in enumerate(keywords, 1):
        print(f"\n{'='*60}")
        print(f"🔍 [{i}/{len(keywords)}] Từ khóa: {keyword}")
        print(f"{'='*60}")

        edge_options = Options()
        edge_options.add_argument(f"--user-data-dir={profile_path}")
        edge_options.add_argument(f"--profile-directory={profile_name}")
        edge_options.add_argument("--window-size=1280,900")
        edge_options.add_argument("--disable-blink-features=AutomationControlled")
        edge_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        edge_options.add_experimental_option('useAutomationExtension', False)
        edge_options.add_argument("--edge-skip-compat-layer-relaunch")

        # Thiết lập User-Agent
        if mode == "mobile":
            edge_options.add_argument(f"--user-agent={MOBILE_UA}")
            edge_options.add_argument("--window-size=390,844")
        else:
            edge_options.add_argument(f"--user-agent={DESKTOP_UA}")

        driver = None
        try:
            service = Service(EDGE_DRIVER_PATH)
            driver = webdriver.Edge(service=service, options=edge_options)  
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

            print("🌐 Đang vào Bing.com...")
            driver.get("https://www.bing.com")

            # Optional: đồng bộ tài khoản nếu cần
            if use_sync:
                sync_edge_account(driver)

            # Tìm ô tìm kiếm
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "q"))
            )
            search_box.clear()
            search_box.send_keys(keyword)
            search_box.send_keys(Keys.RETURN)

            # Đợi kết quả
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "b_results")))
            time.sleep(2)

            # Click kết quả đầu tiên
            first_result = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#b_results .b_algo h2 a"))
            )
            print(f"📄 Kết quả đầu: {first_result.text}")
            print(f"🔗 URL: {first_result.get_attribute('href')}")
            first_result.click()
            print("⏳ Đang xem trang trong 6s...")
            time.sleep(6)

        except TimeoutException:
            print("⚠️ Timeout – Bing không phản hồi kịp.")
        except NoSuchElementException:
            print("⚠️ Không tìm thấy kết quả.")
        except Exception as e:
            print(f"❌ Lỗi không xác định: {e}")
        finally:
            if driver:
                driver.quit()
            kill_msedge_processes()
            print("🔴 Đã đóng browser.")
            time.sleep(2)


# ========================
# DEMO USAGE
# ========================
def quick_search_example():
    profile_path = r"C:\Users\Admin\AppData\Local\Microsoft\Edge\User Data"
    profiles = [
        "Profile 1", ##
    # 'Profile 3', ##
    # 'Profile 1Profile 3',
        ]

    keywords = [
          "Việt Nam",
        "Canada",
        "Úc",
        "Mỹ",
        "Nhật Bản",
        "Hàn Quốc",
        "Trung Quốc",
        "Thái Lan",
        "Singapore",
        "Malaysia",
        "Indonesia",
        "Ấn Độ",
        "Anh Quốc",
        "Pháp",
        "Đức",
        "Ý",
        "Tây Ban Nha",
        "Bồ Đào Nha",
        "Nga",
        "Brazil",
        "Argentina",
        "Mexico",
        "Ai Cập",
        "Nam Phi",
        "UAE",
        "văn hóa Mỹ",
        "ẩm thực Việt Nam",
        "du lịch Nhật Bản",
        "lễ hội Ấn Độ",
        "kiến trúc Pháp",
        "SEO tools 2024",  
    ]

    # 🔁 Chạy DESKTOP trước, rồi MOBILE 
    print("\n=== 🖥️ DESKTOP SEARCH ===")
    for p in profiles:
        auto_bing_search(keywords, profile_path, p, mode="desktop")

    # print("\n=== 📱 MOBILE SEARCH ===")
    for p in profiles:
        auto_bing_search(keywords, profile_path, p, mode="mobile")


if __name__ == "__main__":
    quick_search_example()
177


=== 🖥️ DESKTOP SEARCH ===

🔍 [1/31] Từ khóa: Việt Nam
🌐 Đang vào Bing.com...
⚠️ Timeout – Bing không phản hồi kịp.
🔴 Đã đóng browser.

🔍 [2/31] Từ khóa: Canada
🌐 Đang vào Bing.com...
📄 Kết quả đầu: Canada - Wikipedia
🔗 URL: https://www.bing.com/ck/a?!&&p=ad24d2aef90b21b49dd56f676504208dcbde934ddfac282251caf7b990cacd95JmltdHM9MTc2OTA0MDAwMA&ptn=3&ver=2&hsh=4&fclid=0df68965-dcef-6828-0a4d-9f53dd8969ea&psq=Canada&u=a1aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvQ2FuYWRh&ntb=1
⏳ Đang xem trang trong 6s...
🔴 Đã đóng browser.

🔍 [3/31] Từ khóa: Úc
🌐 Đang vào Bing.com...
📄 Kết quả đầu: Úc – Wikipedia tiếng Việt
🔗 URL: https://www.bing.com/ck/a?!&&p=10fc9ae0d4e4494f4992c81513e54658f7f8b11ea6ef0310a9ece3763c5f1300JmltdHM9MTc2OTA0MDAwMA&ptn=3&ver=2&hsh=4&fclid=0df68965-dcef-6828-0a4d-9f53dd8969ea&psq=%c3%9ac&u=a1aHR0cHM6Ly92aS53aWtpcGVkaWEub3JnL3dpa2kvJUMzJTlBYw&ntb=1
⏳ Đang xem trang trong 6s...
🔴 Đã đóng browser.

🔍 [4/31] Từ khóa: Mỹ
🌐 Đang vào Bing.com...
📄 Kết quả đầu: Hoa Kỳ – Wikipedia tiếng 

177

## Check proxy list:

In [2]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# import time

# proxy = "84.17.47.150:9002"   # Proxy bạn đưa

# def check_proxy(proxy):
#     chrome_options = Options()
#     chrome_options.add_argument(f"--proxy-server=http://{proxy}")
#     chrome_options.add_argument("--ignore-certificate-errors")
#     chrome_options.add_argument("--disable-blink-features=AutomationControlled")

#     # Tự động tải ChromeDriver
#     driver = webdriver.Chrome(
#         service=Service(ChromeDriverManager().install()),
#         options=chrome_options
#     )

#     try:
#         driver.get("https://api.ipify.org?format=json")
#         time.sleep(3)

#         print("Page Content:", driver.page_source)

#         if proxy.split(":")[0] in driver.page_source:
#             print("✅ Proxy hoạt động OK")
#         else:
#             print("❌ Proxy KHÔNG hoạt động hoặc bị chặn")

#     except Exception as e:
#         print("❌ Lỗi khi kiểm tra proxy:", e)

#     finally:
#         driver.quit()


# check_proxy(proxy)


In [3]:
def count_partitions(m, n):
    """
    Đếm số lượng dãy số thỏa mãn điều kiện:
    t1 + t2 + ... + tn = m
    t1 >= t2 >= ... >= tn
    ti >= 0
 
    Args:
        m (int): Tổng của dãy số.
        n (int): Số lượng phần tử trong dãy.
 
    Returns:
        int: Số lượng dãy số thỏa mãn.
    """
    # dp[i][j] sẽ lưu trữ số cách phân hoạch j thành i phần
    # i là số lượng phần (từ 0 đến n)
    # j là tổng (từ 0 đến m)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
 
    # Điều kiện cơ sở:
    # Có 1 cách để phân hoạch 0 thành i phần (tất cả các phần đều là 0)
    for i in range(n + 1):
        dp[i][0] = 1
 
    # Duyệt qua số lượng phần
    for i in range(1, n + 1):
        # Duyệt qua tổng
        for j in range(1, m + 1):
            # Trường hợp 1: Không sử dụng phần tử i (tn = 0).
            # Tức là ta phân hoạch j thành i-1 phần.
            dp[i][j] = dp[i - 1][j]
 
            # Trường hợp 2: Mỗi phần tử trong i phần đều lớn hơn hoặc bằng 1.
            # Ta trừ 1 từ mỗi phần tử, tổng còn lại là j-i.
            # Và ta vẫn cần phân hoạch thành i phần.
            if j >= i:
                dp[i][j] += dp[i][j - i]
 
    return dp[n][m]

In [4]:
count_partitions(3, 2)

2

In [5]:
def count_partitions(m, n):
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    for i in range(n + 1):
        dp[i][0] = 1
 
    for i in range(1, n + 1):
        for j in range(1, m + 1):
          
            dp[i][j] = dp[i - 1][j]
 
            if j >= i:
                dp[i][j] += dp[i][j - i]
 
    return dp[n][m]

In [6]:
def count_partitions(m, n):
    width = m + 1
    size = (n + 1) * (m + 1) + 1  # +1 để dùng chỉ số từ 1

    # dp[i][j] → dp[i * width + j + 1]
    dp = [0] * size

    # Cơ sở: dp[i][0] = 1
    for i in range(n + 1):
        idx = i * width + 0 + 1
        dp[idx] = 1

    # Duyệt theo số phần
    for i in range(1, n + 1):
        for j in range(1, m + 1):

            # vị trí dp[i][j]
            curr = i * width + j + 1

            # dp[i][j] = dp[i-1][j]
            dp[curr] = dp[(i - 1) * width + j + 1]

            # nếu j >= i thì dp[i][j] += dp[i][j - i]
            if j >= i:
                dp[curr] += dp[i * width + (j - i) + 1]

    return dp[n * width + m + 1]

In [7]:
count_partitions(4, 4)

5